In [1]:
import pandas as pd
import numpy as np

### Step 1  Loading the csv file.  

In [2]:
cols = ['bookID' ,'title',  'authors', 'average_rating', 'isbn', 'isbn13', 'language_code', 'num_pages', 'ratings_count',      
        'text_reviews_count', 'publication_date', 'publisher',  'coauthor']
df = pd.read_csv(r'books_data.csv', names=cols, skiprows=1,engine='python', quotechar=',', on_bad_lines='warn')

#### Note we have used quoting charcter ',' to prevent quoting issues.
####  Also engine is python. It is used when no of columns are variable.

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   bookID              11127 non-null  int64 
 1   title               11127 non-null  object
 2   authors             11127 non-null  object
 3   average_rating      11127 non-null  object
 4   isbn                11127 non-null  object
 5   isbn13              11127 non-null  object
 6   language_code       11127 non-null  object
 7   num_pages           11127 non-null  object
 8   ratings_count       11127 non-null  int64 
 9   text_reviews_count  11127 non-null  int64 
 10  publication_date    11127 non-null  object
 11  publisher           11127 non-null  object
 12  coauthor            4 non-null      object
dtypes: int64(3), object(10)
memory usage: 1.1+ MB


In [4]:
df.head(2)

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,coauthor
0,1,Harry Potter and the Half-Blood Prince (Harry ...,J.K. Rowling/Mary GrandPré,4.57,0439785960,9780439785969,eng,652,2095690,27591,9/16/2006,Scholastic Inc.,None
1,2,Harry Potter and the Order of the Phoenix (Har...,J.K. Rowling/Mary GrandPré,4.49,0439358078,9780439358071,eng,870,2153167,29221,9/1/2004,Scholastic Inc.,None


###  Step 2 Remove dirty data for four rows.

#### As we can see the last column is problematic. Its is actually a co author name. But this co author name is present for only four rows in the datsset. 

#### Lets examine those four rows. 

In [5]:
dirty = df.loc[~df['coauthor'].isnull(), : ]

In [6]:
dirty

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,coauthor
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner,Jr./Sam B. Warner,3.58,0674842111,9780674842113,en-US,236,61,6,4/20/2004,Harvard University Press
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net,one of the founding members of this Tolkien w...,3.58,1593600119,9781593600112,eng,400,26,4,4/6/2004,Cold Spring Press\t
5877,22128,Patriots (The Coming Collapse),James Wesley,Rawles,3.63,156384155X,9781563841552,eng,342,38,4,1/15/1999,Huntington House Publishers
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown,Son & Ferguson,0.00,0851742718,9780851742717,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.


#### The average rating for these four rows is actaully a author name. 

In [7]:
coauthor = dirty.loc[:, 'average_rating']
coauthor = coauthor.rename("author2")

#### We will shift every other column to the left and we call this right side of data frame. 

In [8]:
right = dirty.loc[:, 'average_rating':'coauthor'].shift(periods=-1, axis=1)

#### All the other columns before author and taken as left. 

In [9]:
left = dirty.loc[:, :'authors']

#### We will merge the co author with author.  

In [10]:
left['authors'] = left['authors'] + coauthor

#### The left side will look like this. 

In [11]:
left

,bookID,title,authors
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner Jr./Sam B. Warner
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net one o...
5877,22128,Patriots (The Coming Collapse),James Wesley Rawles
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown Son & Ferguson


#### The Right side

In [12]:
right

,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,coauthor
3348,3.58,0674842111,9780674842113,en-US,236,61,6,4/20/2004,Harvard University Press,None
4702,3.58,1593600119,9781593600112,eng,400,26,4,4/6/2004,Cold Spring Press\t,None
5877,3.63,156384155X,9781563841552,eng,342,38,4,1/15/1999,Huntington House Publishers,None
8979,0.00,0851742718,9780851742717,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.,None


####  We will combine them. and call it clean. 

In [13]:
clean = pd.concat([left,  right, coauthor], axis=1)

In [14]:
clean = clean.drop(columns= ['coauthor'])

In [15]:
clean

,bookID,title,authors,average_rating,isbn,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,author2
3348,12224,Streetcar Suburbs: The Process of Growth in Bo...,Sam Bass Warner Jr./Sam B. Warner,3.58,0674842111,9780674842113,en-US,236,61,6,4/20/2004,Harvard University Press,Jr./Sam B. Warner
4702,16914,The Tolkien Fan's Medieval Reader,David E. Smith (Turgon of TheOneRing.net one o...,3.58,1593600119,9781593600112,eng,400,26,4,4/6/2004,Cold Spring Press\t,one of the founding members of this Tolkien w...
5877,22128,Patriots (The Coming Collapse),James Wesley Rawles,3.63,156384155X,9781563841552,eng,342,38,4,1/15/1999,Huntington House Publishers,Rawles
8979,34889,Brown's Star Atlas: Showing All The Bright Sta...,Brown Son & Ferguson,0.00,0851742718,9780851742717,eng,49,0,0,5/1/1977,Brown Son & Ferguson Ltd.,Son & Ferguson


#### We will replace the dirty rows with this newly created clean data frame. 

In [16]:
df.loc[clean.index, :] = clean

### Step 3 We will remove 'coauthor', 'bookID', 'isbn', 'isbn13' columns as they do not make any sense wrt to rating prediction. 

In [17]:
df = df.drop(columns= ['coauthor', 'bookID', 'isbn', 'isbn13'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               11127 non-null  object
 1   authors             11127 non-null  object
 2   average_rating      11127 non-null  object
 3   language_code       11127 non-null  object
 4   num_pages           11127 non-null  object
 5   ratings_count       11127 non-null  int64 
 6   text_reviews_count  11127 non-null  object
 7   publication_date    11127 non-null  object
 8   publisher           11127 non-null  object
dtypes: int64(1), object(8)
memory usage: 782.5+ KB


### Step 4 We will also convert the numeric columns to int/ float values. 

In [19]:
df['average_rating'] = df['average_rating'].astype(np.float32)
df['num_pages'] = df['num_pages'].astype(np.int32)
df['ratings_count'] = df['ratings_count'].astype(np.int32)
df['text_reviews_count'] = df['text_reviews_count'].astype(np.int32)

In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               11127 non-null  object 
 1   authors             11127 non-null  object 
 2   average_rating      11127 non-null  float32
 3   language_code       11127 non-null  object 
 4   num_pages           11127 non-null  int32  
 5   ratings_count       11127 non-null  int32  
 6   text_reviews_count  11127 non-null  int32  
 7   publication_date    11127 non-null  object 
 8   publisher           11127 non-null  object 
dtypes: float32(1), int32(3), object(5)
memory usage: 608.6+ KB


#### Let's just check our plan worked or not! 

In [21]:
df.iloc[3348, :]

title                 Streetcar Suburbs: The Process of Growth in Bo...
authors                               Sam Bass Warner Jr./Sam B. Warner
average_rating                                                     3.58
language_code                                                     en-US
num_pages                                                           236
ratings_count                                                        61
text_reviews_count                                                    6
publication_date                                              4/20/2004
publisher                                      Harvard University Press
Name: 3348, dtype: object

### Step 4 It did Now let's convert dates 

In [22]:
dates = pd.to_datetime(df.publication_date, format="%m/%d/%Y", errors="coerce")

In [23]:
dates[dates.isnull()]

8180    NaT
11098   NaT
Name: publication_date, dtype: datetime64[ns]

#### These two rows contain dirty values for dates. 

In [24]:
df.publication_date[8180]

'11/31/2000'

In [25]:
df.publication_date[11098]

'6/31/1982'

#### Let us correct it. And parse again. 

In [26]:
df.iloc[8180, 7] = '11/30/2000' 
df.iloc[11098, 7] = '6/30/1982'

df.publication_date = pd.to_datetime(df.publication_date, format="%m/%d/%Y", errors="raise")

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               11127 non-null  object        
 1   authors             11127 non-null  object        
 2   average_rating      11127 non-null  float32       
 3   language_code       11127 non-null  object        
 4   num_pages           11127 non-null  int32         
 5   ratings_count       11127 non-null  int32         
 6   text_reviews_count  11127 non-null  int32         
 7   publication_date    11127 non-null  datetime64[ns]
 8   publisher           11127 non-null  object        
dtypes: datetime64[ns](1), float32(1), int32(3), object(4)
memory usage: 608.6+ KB


#### Now we wil subtract datetime object with 01/01/1970 to obtain an integer. 

In [28]:
df.publication_date = (df.publication_date - pd.to_datetime('1970-01-01', format='%Y-%m-%d')).dt.days

### Step5 Now Let's  separate out all the co authors. Co authors are separted by a /. 

In [29]:
author_name=[]
for i in range(1, df["authors"].str.split("/", expand=True).shape[1] +1):
    author_name.append(f"Author_{i}") 
    

In [30]:
splitted = df["authors"].str.split("/", expand=True)

In [31]:
splitted.columns = author_name

In [32]:
df  = pd.concat((df, splitted), axis=1)

In [33]:
 df = df.drop(columns= ['authors'])

In [34]:
df.head(2)

,title,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Author_1,Author_2,...,Author_42,Author_43,Author_44,Author_45,Author_46,Author_47,Author_48,Author_49,Author_50,Author_51
0,Harry Potter and the Half-Blood Prince (Harry ...,4.57,eng,652,2095690,27591,13407,Scholastic Inc.,J.K. Rowling,Mary GrandPré,...,None,None,None,None,None,None,None,None,None,None
1,Harry Potter and the Order of the Phoenix (Har...,4.49,eng,870,2153167,29221,12662,Scholastic Inc.,J.K. Rowling,Mary GrandPré,...,None,None,None,None,None,None,None,None,None,None


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11127 entries, 0 to 11126
Data columns (total 59 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               11127 non-null  object 
 1   average_rating      11127 non-null  float32
 2   language_code       11127 non-null  object 
 3   num_pages           11127 non-null  int32  
 4   ratings_count       11127 non-null  int32  
 5   text_reviews_count  11127 non-null  int32  
 6   publication_date    11127 non-null  int64  
 7   publisher           11127 non-null  object 
 8   Author_1            11127 non-null  object 
 9   Author_2            4564 non-null   object 
 10  Author_3            1485 non-null   object 
 11  Author_4            479 non-null    object 
 12  Author_5            257 non-null    object 
 13  Author_6            192 non-null    object 
 14  Author_7            138 non-null    object 
 15  Author_8            117 non-null    object 
 16  Auth

In [36]:
df.columns

Index(['title', 'average_rating', 'language_code', 'num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher',
       'Author_1', 'Author_2', 'Author_3', 'Author_4', 'Author_5', 'Author_6',
       'Author_7', 'Author_8', 'Author_9', 'Author_10', 'Author_11',
       'Author_12', 'Author_13', 'Author_14', 'Author_15', 'Author_16',
       'Author_17', 'Author_18', 'Author_19', 'Author_20', 'Author_21',
       'Author_22', 'Author_23', 'Author_24', 'Author_25', 'Author_26',
       'Author_27', 'Author_28', 'Author_29', 'Author_30', 'Author_31',
       'Author_32', 'Author_33', 'Author_34', 'Author_35', 'Author_36',
       'Author_37', 'Author_38', 'Author_39', 'Author_40', 'Author_41',
       'Author_42', 'Author_43', 'Author_44', 'Author_45', 'Author_46',
       'Author_47', 'Author_48', 'Author_49', 'Author_50', 'Author_51'],
      dtype='object')

###  Step6 Lets calculate top 5 authors with highest rating

In [37]:
df.sort_values('average_rating', ascending=False).head(2)

,title,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher,Author_1,Author_2,...,Author_42,Author_43,Author_44,Author_45,Author_46,Author_47,Author_48,Author_49,Author_50,Author_51
5650,Colossians and Philemon: A Critical and Exeget...,5.0,eng,512,1,0,13124,T&T Clark Int'l,R. McL. Wilson,None,...,None,None,None,None,None,None,None,None,None,None
1243,Middlesex Borough (Images of America: New Jersey),5.0,eng,128,2,0,12128,Arcadia Publishing,Middlesex Borough Heritage Committee,None,...,None,None,None,None,None,None,None,None,None,None


####  We should take atleast 1000 reviews to be sure. 


In [38]:
eligible = df[df.ratings_count > 500]
top_authors_total = eligible.sort_values('average_rating', ascending=False)


#### Get top 20 authors and co authors from them. 

In [39]:
n_coauthors = 10
top_authors = set()
for i in range(1, n_coauthors+1):
    top_authors = top_authors.union(set(top_authors_total.head(20)['Author_'+ str(i)]))

In [40]:
try:
    top_authors.remove(None)
    top_authors.remove('Anonymous')
except KeyError:
    pass

In [41]:
top_authors = list(top_authors)

In [42]:
top_authors

['Joe Yamazaki',
 'Bill Watterson',
 'Kelly Jones',
 'Frank Darabont',
 'Maya Angelou',
 'Neil Gaiman',
 'Chris Bachalo',
 'Matt Thorn',
 'Mike Dringenberg',
 'Kaori Inoue',
 'Izumi Evers',
 'Don Rosa',
 'J.K. Rowling',
 'Michael Zulli',
 'David Allen Sibley',
 'Hayao Miyazaki',
 'Walden Wong',
 'Colleen Doran',
 'Daniel Vozzo',
 'Arthur Conan Doyle',
 'Robert B. Leighton',
 'Matthew L. Sands',
 'Steve Parkhouse',
 'Stephen King',
 'Leslie S. Klinger',
 'Charles Vess',
 "Patrick O'Brian",
 'Mary GrandPré',
 'Malcolm Jones III',
 'Richard P. Feynman']

###  Step6 We calculate top publishers as above

In [43]:
top_publishers_total = eligible.sort_values('average_rating', ascending=False)

In [44]:
top_publishers = list(set(top_publishers_total.head(20)['publisher']))

In [45]:
top_publishers

['Zondervan Publishing House',
 'VIZ Media',
 'Scholastic',
 'Alfred A. Knopf',
 'W. W. Norton  Company',
 'Addison Wesley Publishing Company',
 'Modern Library',
 'Gemstone Publishing',
 'Andrews McMeel Publishing',
 'W. W. Norton & Company',
 'Vertigo',
 'Newmarket Press']

### Step7 Let us construct two columns 
### 1. Whether given author belongs to a group of top 20 rated authors.
### 2. 1. Whether given publisher belongs to a group of top 20 rated publisher.
### Remove bogus columns as well.

In [46]:
def check_top_publishers(row, publisher_list):
    if not row in publisher_list:
        return False
    return True

df['publisher'] = df.loc[:,['publisher']].applymap(check_top_publishers, publisher_list=top_publishers)

def check_top_authors(row, authors_list):
    if not row in authors_list:
        return False
    return True

df[author_name] = df[author_name].applymap(check_top_authors, authors_list=top_authors)

df['is_top_20_author'] = False
df['is_top_20_publisher'] = df.publisher
for auth in author_name:
    df['is_top_20_author'] = df['is_top_20_author'] | df[auth]

df.drop(author_name + ['publisher', 'title'], inplace=True, axis=1)

#### Note we have removed author name, publisher name and title from original data. 

In [47]:
df.head(3)

,average_rating,language_code,num_pages,ratings_count,text_reviews_count,publication_date,is_top_20_author,is_top_20_publisher
0,4.57,eng,652,2095690,27591,13407,True,False
1,4.49,eng,870,2153167,29221,12662,True,False
2,4.42,eng,352,6333,244,12357,True,True


In [48]:
len(list(set(df.language_code)))

27

In [56]:
ratings_95 = np.percentile(df.ratings_count.values, 5)

In [58]:
ratings_95

8.0

In [51]:
df.ratings_count.values

array([2095690, 2153167,    6333, ...,     820,     769,     113])

In [52]:
#Replace all english related language(en-US,en-GB,en-CA,enm) with one global code:eng
df['language_code']=df['language_code'].replace(['en-US','en-GB','en-CA','eng','enm'],'eng')

In [53]:
np.unique(df.language_code)

array(['ale', 'ara', 'eng', 'fre', 'ger', 'gla', 'glg', 'grc', 'ita',
       'jpn', 'lat', 'msa', 'mul', 'nl', 'nor', 'por', 'rus', 'spa',
       'srp', 'swe', 'tur', 'wel', 'zho'], dtype=object)

In [54]:
# df['is_top_20_author'] = pd.to_numeric(df['is_top_20_author'])

df.is_top_20_author = df.is_top_20_author.replace({True: 1, False: 0})
df.is_top_20_publisher = df.is_top_20_publisher.replace({True: 1, False: 0})


In [55]:
df.dtypes

average_rating         float32
language_code           object
num_pages                int32
ratings_count            int32
text_reviews_count       int32
publication_date         int64
is_top_20_author         int64
is_top_20_publisher      int64
dtype: object

In [79]:
!pip install category_encoders

     ---------------------------------------- 81.9/81.9 kB 4.5 MB/s eta 0:00:00


In [59]:
from category_encoders import BinaryEncoder

In [60]:
binary_encoder = BinaryEncoder(cols=['language_code'])
df = binary_encoder.fit_transform(df)

In [61]:
df

,average_rating,language_code_0,language_code_1,language_code_2,language_code_3,language_code_4,num_pages,ratings_count,text_reviews_count,publication_date,is_top_20_author,is_top_20_publisher
0,4.57,0,0,0,0,1,652,2095690,27591,13407,1,0
1,4.49,0,0,0,0,1,870,2153167,29221,12662,1,0
2,4.42,0,0,0,0,1,352,6333,244,12357,1,1
3,4.56,0,0,0,0,1,435,2339585,36325,12539,1,0
4,4.78,0,0,0,0,1,2690,41428,164,12674,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11122,4.06,0,0,0,0,1,512,156,20,12773,0,0
11123,4.08,0,0,0,0,1,635,783,56,6909,0,0
11124,3.96,0,0,0,0,1,415,820,95,8613,0,0
11125,3.72,0,0,0,0,1,434,769,139,13571,0,0


###  Lets filter outer data on the basis of 95 percentile rule. 

In [63]:
df= df.loc[df.loc[:, 'ratings_count'] > 8]

In [64]:
df.shape

(10513, 12)

###  Lets separate out X and y

In [65]:
X = df.iloc[:,1:]
y = df.iloc[:,1]

###  Let's apply min max scaler 

In [66]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X = scaler.fit_transform(X)


### Apply linear regression 

In [68]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression

 
k = 5
kf = KFold(n_splits=k, random_state=None)
lr_model= LinearRegression()

result = cross_val_score(lr_model , X, y, scoring='r2', cv = kf)

print("Avg accuracy: {}".format(result.mean()))

Avg accuracy: 0.6


### We see we have fair accuracy just by linear regression.  [Ref](https://stephenallwright.com/good-accuracy-score/)